In [1]:
from rdkit.Chem.rdmolfiles import MolFromPDBFile
from rdkit.Chem.rdchem import Mol
from rdkit.Chem import AllChem

import numpy as np
import rdkit.Chem as Chem
from rdkit.Chem import AddHs, AssignStereochemistry, HybridizationType, ChiralType, BondStereo, MolFromMol2File
from rdkit.Chem.AllChem import ComputeGasteigerCharges
import os
import sys
sys.path.append("../../")
from src.data.pocket_utils import get_atom_coordinates, find_pocket_atoms_RDKit
from src.data.utils import pdb_to_rdkit_mol, mol2_to_rdkit_mol, get_vdw_radius, add_charges_to_molecule, get_node_features, get_edge_features, extract_charges_from_mol2
from src.utils.constants import ES_THRESHOLD


In [2]:
def ionic_interaction(mol: Chem.Mol, atom_1: int, atom_2: int):
    """
    Determine if there is an electrostatic interaction between atom 1 and atom 2 based on whether the coulombic interaction
    between the two atoms is less than or equal to some threshold imported from src/utils/constants.py.

    Parameters:
    mol (Chem.Mol): The RDKit molecule. NOTE: We assume that the molecule has _TriposAtomCharges charges assigned to the atoms.
    atom_1 (int): The index of the first atom.
    atom_2 (int): The index of the second atom.
    
    Returns:
    tuple: (0, 0, 0, 0, 0) if there is an electrostatic interaction, None otherwise 

    Conversion Factor Calculation:
    The Coulombic interaction energy between two point charges q1 and q2 separated by a distance r in vacuum is given by:

        E = k * q1 * q2 / r

    Where:
    - E is the energy in Joules (J).
    - k is Coulomb's constant, 8.987 x 10^9 N m^2 C^-2.
    - q1 and q2 are the charges in Coulombs (C).
    - r is the distance in meters (m).

    To convert this energy into kJ/mol, the following conversions are applied:
    - 1 e (elementary charge) = 1.602 x 10^-19 C
    - Avogadro's number (N_A) = 6.022 x 10^23 mol^-1
    - 1 Angstrom (Å) = 10^-10 meters (m)

    Therefore, the conversion factor from (charge_1 * charge_2) / distance to kJ/mol is:

        Conversion factor = (k * N_A * (1.602 x 10^-19)^2) * 10^10

    Plugging in the values:

        Conversion factor = (8.987 x 10^9 * 6.022 x 10^23 * (1.602 x 10^-19)^2) * 10^10
                          ≈ 1388.93 kJ mol^-1 Å^-1
    """
    # Constants
    CONVERSION_FACTOR_KJ = 1388.93  # kJ mol^-1 Å^-1

    # Get atomic charges using Gasteiger charges or another suitable method
    charge_1 = float(mol.GetAtomWithIdx(atom_1).GetProp('_TriposAtomCharges'))
    charge_2 = float(mol.GetAtomWithIdx(atom_2).GetProp('_TriposAtomCharges'))
    
    # Get the distance between the two atoms
    conf = mol.GetConformer()
    pos_1 = conf.GetAtomPosition(atom_1)
    pos_2 = conf.GetAtomPosition(atom_2)
    distance = pos_1.Distance(pos_2)

    coulombic_interaction_kj = (charge_1 * charge_2 / distance) * CONVERSION_FACTOR_KJ

    # Check if the Coulombic interaction is less than or equal to the threshold
    if abs(coulombic_interaction_kj) >= ES_THRESHOLD:
        return (0, 0, 0, 0, 0)
    else:
        return None

In [3]:
def calculate_gasteiger_charges(mol):
    AllChem.ComputeGasteigerCharges(mol)
    for atom in mol.GetAtoms():
        atom.SetProp('_TriposAtomCharges', str(atom.GetDoubleProp('_GasteigerCharge')))
    return mol


In [5]:
# Define test cases
test_cases = [("CCCCCCCCCC=C", 0, 10), # expected output: None
              ("C=C", 0, 1), # expected output:  None
              ("CC(=O)O", 2, 3) # expected output: (0, 0, 0, 0, 0)
              ]

# Initialize RDKit molecules and conformers
molecules = []
for smiles, a1, a2 in test_cases:
    mol = Chem.MolFromSmiles(smiles)
    AllChem.EmbedMolecule(mol)
    mol = calculate_gasteiger_charges(mol)
    molecules.append((mol, a1, a2))

# Run test cases
for mol, atom_1, atom_2 in molecules:
    # Replace this with the actual function call
    result = ionic_interaction(mol, atom_1, atom_2)
    print(f"SMILES: {Chem.MolToSmiles(mol)}, Atoms: ({atom_1}, {atom_2}) -> Result: {result}")


SMILES: C=CCCCCCCCCC, Atoms: (0, 10) -> Result: None
SMILES: C=C, Atoms: (0, 1) -> Result: None
SMILES: CC(=O)O, Atoms: (2, 3) -> Result: (0, 0, 0, 0, 0)


[18:17:04] Molecule does not have explicit Hs. Consider calling AddHs()
[18:17:04] Molecule does not have explicit Hs. Consider calling AddHs()
[18:17:04] Molecule does not have explicit Hs. Consider calling AddHs()


In [7]:
complex_mol = mol2_to_rdkit_mol("../../test_data/pdb/1a0q/1a0q_complex_charged.mol2", sanitize = False) #/Users/tsachmackey/Documents/Summer 2024/Research /Batista project/AffinityNet/test_data/pdb/1a0q
protein_mol = pdb_to_rdkit_mol("../../test_data/pdb/1a0q/1a0q_protein.pdb") #/Users/tsachmackey/Documents/Summer 2024/Research /Batista project/AffinityNet/test_data/pdb/1a0q
num_atoms_in_protein = len(protein_mol.GetAtoms())
charges = extract_charges_from_mol2("../../test_data/pdb/1a0q/1a0q_complex_charged.mol2")

In [8]:
mol = complex_mol

AssignStereochemistry(mol, force=True, cleanIt=True)

protein_or_ligand_ids = [-1 if atom.GetIdx() < num_atoms_in_protein else 1 for atom in mol.GetAtoms()]

add_charges_to_molecule(mol, charges)

pocket_atom_indices = find_pocket_atoms_RDKit(mol, protein_or_ligand_ids, num_atoms_in_protein)

edge_features, edge_indices = get_edge_features(mol, pocket_atom_indices, pairwise_function=ionic_interaction)

  0%|          | 0/3071 [00:51<?, ?it/s]


KeyboardInterrupt: 

In [39]:
len(pocket_atom_indices)

NameError: name 'pocket_atom_indices' is not defined